In [19]:
import os

os.environ["WANDB_DISABLED"] = "true"

In [20]:
import random
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import re

# Fantasy-themed character attributes
RACES = ["Human", "Elf", "Dwarf", "Halfling", "Orc", "Gnome", "Tiefling", "Dragonborn"]
CLASSES = ["Warrior", "Mage", "Rogue", "Cleric", "Paladin", "Ranger", "Bard", "Necromancer"]
ALIGNMENTS = ["Lawful Good", "Neutral Good", "Chaotic Good", "Lawful Neutral",
              "True Neutral", "Chaotic Neutral", "Lawful Evil", "Neutral Evil", "Chaotic Evil"]


In [21]:
# Load pre-trained model and tokenizer globally
TOKENIZER = GPT2Tokenizer.from_pretrained("gpt2")
MODEL = GPT2LMHeadModel.from_pretrained("gpt2")

# Set pad token to fix warnings
TOKENIZER.pad_token = TOKENIZER.eos_token
MODEL.config.pad_token_id = MODEL.config.eos_token_id

In [22]:
def generate_random_name():
    """Generate a random fantasy name"""
    consonants = "bcdfghjklmnpqrstvwxyz"
    vowels = "aeiou"
    name_length = random.randint(3, 8)
    name = ""

    for i in range(name_length):
        if i % 2 == 0:
            name += random.choice(consonants)
        else:
            name += random.choice(vowels)

    return name.capitalize()

In [23]:
def generate_random_character():
    """Generate random character attributes"""
    character = {
        "name": generate_random_name(),
        "race": random.choice(RACES),
        "class": random.choice(CLASSES),
        "alignment": random.choice(ALIGNMENTS),
        "age": random.randint(18, 500),  # wide range to accommodate different races
        "gender": random.choice(["Male", "Female", "Non-binary"])
    }
    return character

In [24]:
def collect_user_input():
    """Collect character information from user"""
    print("\n=== Character Creation ===")
    character = {}

    character["name"] = input("Enter character name: ")

    print("\nChoose a race:")
    for i, race in enumerate(RACES, 1):
        print(f"{i}. {race}")
    race_choice = int(input("Enter number: "))
    character["race"] = RACES[race_choice-1]

    print("\nChoose a class:")
    for i, cls in enumerate(CLASSES, 1):
        print(f"{i}. {cls}")
    class_choice = int(input("Enter number: "))
    character["class"] = CLASSES[class_choice-1]

    print("\nChoose an alignment:")
    for i, alignment in enumerate(ALIGNMENTS, 1):
        print(f"{i}. {alignment}")
    alignment_choice = int(input("Enter number: "))
    character["alignment"] = ALIGNMENTS[alignment_choice-1]

    character["age"] = int(input("\nEnter character age: "))

    print("\nChoose gender:")
    print("1. Male")
    print("2. Female")
    print("3. Non-binary")
    gender_choice = int(input("Enter number: "))
    genders = ["Male", "Female", "Non-binary"]
    character["gender"] = genders[gender_choice-1]

    return character

In [25]:
def clean_text(text):
    """Clean generated text from URLs, HTML, and non-fantasy references"""
    # Remove URLs
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'www\.\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<[^>]*>', '', text)

    # Remove references to sci-fi terms or modern technology
    sci_fi_terms = ['jedi', 'han solo', 'droid', 'facebook', 'http', 'iframe']
    for term in sci_fi_terms:
        text = re.sub(r'(?i)' + term, '', text)

    # Remove any text after signs of text deterioration
    cutoff_markers = ['#', '---', '===', '...com']
    for marker in cutoff_markers:
        if marker in text:
            text = text.split(marker)[0]

    return text.strip()

In [26]:
def generate_backstory(character):
    """Generate character backstory using GPT-2"""
    # Create a more guided prompt for fantasy-themed backstories
    prompt = f"""Generate a fantasy backstory for:
Character Name: {character['name']}
Race: {character['race']}
Class: {character['class']}
Alignment: {character['alignment']}
Age: {character['age']}
Gender: {character['gender']}

Write a coherent fantasy backstory about this character's origins, motivations, and key life events. The story should be consistent with their alignment and class.

Backstory:
"""

    # Encode the prompt with attention mask
    inputs = TOKENIZER(prompt, return_tensors="pt", padding=True)
    attention_mask = inputs['attention_mask']

    # Generate text
    with torch.no_grad():
        outputs = MODEL.generate(
            inputs['input_ids'],
            attention_mask=attention_mask,
            max_length=inputs['input_ids'].shape[1] + 250,  # Limiting length to avoid cutoffs
            num_return_sequences=1,
            temperature=0.9,
            top_k=50,
            top_p=0.92,
            repetition_penalty=1.2,
            do_sample=True,
            no_repeat_ngram_size=3  # Avoid repeating the same phrases
        )

    # Decode the generated text
    generated_text = TOKENIZER.decode(outputs[0], skip_special_tokens=True)

    # Extract just the backstory part and clean it
    try:
        backstory = generated_text.split("Backstory:")[1].strip()
        # Clean the backstory from any URLs, HTML, or non-fantasy references
        backstory = clean_text(backstory)
    except IndexError:
        # If splitting fails, use the whole text but still clean it
        backstory = clean_text(generated_text)

    return backstory


In [27]:
def format_character_sheet(character, backstory):
    """Format the character sheet"""
    sheet = "\n" + "="*60 + "\n"
    sheet += f"CHARACTER SHEET\n"
    sheet += "="*60 + "\n\n"

    sheet += f"Name: {character['name']}\n"
    sheet += f"Race: {character['race']}\n"
    sheet += f"Class: {character['class']}\n"
    sheet += f"Alignment: {character['alignment']}\n"
    sheet += f"Age: {character['age']}\n"
    sheet += f"Gender: {character['gender']}\n\n"

    sheet += "BACKSTORY:\n"
    sheet += "-"*60 + "\n"
    sheet += backstory + "\n"
    sheet += "-"*60 + "\n"

    return sheet

In [28]:
def main():
    print("Welcome to the RPG Character Generator!")

    try:
        while True:
            print("\nOptions:")
            print("1. Create your own character")
            print("2. Generate a random character")
            print("3. Exit")

            choice = input("\nEnter your choice (1-3): ")

            if choice == "1":
                try:
                    character = collect_user_input()
                    print("\nGenerating character backstory...")
                    backstory = generate_backstory(character)
                    character_sheet = format_character_sheet(character, backstory)
                    print(character_sheet)
                except Exception as e:
                    print(f"An error occurred while creating your character: {e}")
                    print("Let's try again.")

            elif choice == "2":
                try:
                    print("\nGenerating random character...")
                    character = generate_random_character()
                    backstory = generate_backstory(character)
                    character_sheet = format_character_sheet(character, backstory)
                    print(character_sheet)
                except Exception as e:
                    print(f"An error occurred while generating a random character: {e}")
                    print("Let's try again.")

            elif choice == "3":
                print("Thank you for using the RPG Character Generator!")
                break

            else:
                print("Invalid choice. Please try again.")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        print("Please check your installation of the required packages.")
        print("You may need to run: pip install torch transformers")


In [29]:
if __name__ == "__main__":
    main()

Welcome to the RPG Character Generator!

Options:
1. Create your own character
2. Generate a random character
3. Exit

Enter your choice (1-3): 3
Thank you for using the RPG Character Generator!
